In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [33]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("gender_submission.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [6]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [7]:
train["Title"] = train["Name"].str.extract(r',\s*([^.]+)\.',expand = False)

In [8]:
train["Title"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [9]:
train["Age"] = train["Age"].fillna(train.groupby("Title")["Age"].transform("mean"))
train["Embarked"] = train["Embarked"].fillna(train.groupby("Title")["Embarked"].transform(lambda x:x.fillna(x.mode()[0])))
train["Cabin"] = train["Cabin"].apply(lambda x: 0 if pd.isna(x) else 1)

In [10]:
test["Title"] = test["Name"].str.extract(r',\s*([^.]+)\.',expand = False)

In [11]:
test["Age"] = test["Age"].fillna(test.groupby("Title")["Age"].transform("mean"))
test["Cabin"] = test["Cabin"].apply(lambda x:0 if pd.isna(x) else 1)
test["Fare"] = test["Fare"].fillna(test["Fare"].mean())
test["Age"] = test["Age"].fillna(test["Age"].mean())

In [12]:
train.drop(columns = "Name",inplace = True)
test.drop(columns = "Name",inplace = True)

In [13]:
train.drop(columns = "PassengerId",inplace = True)
test.drop(columns = "PassengerId",inplace = True)

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train["Sex"] = le.fit_transform(train["Sex"])
le = LabelEncoder()
test["Sex"] = le.fit_transform(test["Sex"])

In [15]:
from sklearn.preprocessing import OneHotEncoder
enocde = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
train_col = enocde.fit_transform(train[["Embarked"]])
train_encoded = pd.DataFrame(train_col,columns = enocde.get_feature_names_out(["Embarked"]))
test_col = enocde.transform(test[["Embarked"]])
test_encoded = pd.DataFrame(test_col,columns = enocde.get_feature_names_out(["Embarked"]))

In [16]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,0,3,1,22.0,1,0,A/5 21171,7.2500,0,S,Mr
1,1,1,0,38.0,1,0,PC 17599,71.2833,1,C,Mrs
2,1,3,0,26.0,0,0,STON/O2. 3101282,7.9250,0,S,Miss
3,1,1,0,35.0,1,0,113803,53.1000,1,S,Mrs
4,0,3,1,35.0,0,0,373450,8.0500,0,S,Mr


In [17]:
train_encoded = pd.concat([train.drop("Embarked",axis =1),train_encoded],axis =1)
test_encoded = pd.concat([test.drop("Embarked",axis = 1),test_encoded],axis = 1)

In [18]:
train_encoded.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Title,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,A/5 21171,7.2500,0,Mr,0.0,0.0,1.0
1,1,1,0,38.0,1,0,PC 17599,71.2833,1,Mrs,1.0,0.0,0.0
2,1,3,0,26.0,0,0,STON/O2. 3101282,7.9250,0,Miss,0.0,0.0,1.0
3,1,1,0,35.0,1,0,113803,53.1000,1,Mrs,0.0,0.0,1.0
4,0,3,1,35.0,0,0,373450,8.0500,0,Mr,0.0,0.0,1.0


In [19]:
train_encoded = train_encoded.drop(columns=["Ticket"])
test_encoded = test_encoded.drop(columns=["Ticket"])

In [20]:
ecoder = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
train_col = enocde.fit_transform(train[["Title"]])
train_encoded = pd.DataFrame(train_col,columns = enocde.get_feature_names_out(["Title"]))
test_col = enocde.transform(test[["Title"]])
test_encoded = pd.DataFrame(test_col,columns = enocde.get_feature_names_out(["Title"]))

In [21]:
train_encoded = pd.concat([train.drop("Title",axis =1),train_encoded],axis =1)
test_encoded = pd.concat([test.drop("Title",axis = 1),test_encoded],axis = 1)

In [22]:
train_encoded.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,0,3,1,22.0,1,0,A/5 21171,7.2500,0,S,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0,38.0,1,0,PC 17599,71.2833,1,C,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,3,0,26.0,0,0,STON/O2. 3101282,7.9250,0,S,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,0,35.0,1,0,113803,53.1000,1,S,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,3,1,35.0,0,0,373450,8.0500,0,S,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [23]:
train_encoded = train_encoded.drop(columns = ["Embarked","Ticket"])

In [24]:
test_encoded = test_encoded.drop(columns = ["Embarked","Ticket"])

In [25]:
train_encoded.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Title_Capt,Title_Col,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,0,3,1,22.0,1,0,7.2500,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0,38.0,1,0,71.2833,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,3,0,26.0,0,0,7.9250,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,0,35.0,1,0,53.1000,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,3,1,35.0,0,0,8.0500,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [26]:
y = train_encoded["Survived"]
train_encoded.drop(columns=["Survived"],inplace = True)

In [27]:
from sklearn.preprocessing import RobustScaler
scale = RobustScaler()
train_encoded = scale.fit_transform(train_encoded)
test_encoded = scale.transform(test_encoded)

In [29]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(train_encoded,y)


,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the curre

In [31]:
test_predict = classifier.predict(test_encoded)

In [34]:
submission = pd.DataFrame({"PassengerId":test["PassengerId"],"Survived" : test_predict})

In [35]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0


In [36]:
submission.to_csv("Submission.csv",index = False)